In [2]:
# import libraries for loading dataset
import pandas as pd
import numpy as np
# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
%matplotlib inline
plt.style.use('dark_background')
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# loading the data set
df = pd.read_csv("/Users/sudachk/PacktPublishing/DataLabelling/Ch01/adult.csv", encoding='latin-1')

In [5]:
df.head(5)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [8]:
x = df.iloc[:,:-1]
x

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States


In [11]:

from snorkel.labeling import labeling_function


In [10]:
y=df['income']
y

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
32556    <=50K
32557    <=50K
32558     >50K
32559    <=50K
32560    <=50K
Name: income, Length: 32561, dtype: object

In [40]:

# income_high indicates income>50 K 
# income_low indicates income<50 k
# ABSTAIN indicates income not falls in any range for that observation data point
ABSTAIN = -1
income_high = 1
income_low =0



In [62]:
# Based on exploratory Bi-variate analysis of age and income,we comeup with heurstics rule that
# if age range is between 28-58 then income is greate than 50 K else less than 50 K

@labeling_function()
def age(record):
    if record['age'] < 28 and record['age'] > 58:
     return income_low
    elif record['age'] >28 and record['age']< 58:
     return income_high
    else:
     return	ABSTAIN


In [63]:
# Based on Bi-variate analysis of Education and income,we comeup with heuristic rule that 
# if Education is Bacherlors or Masters then income is > 50 K otherwise income < 50 K
@labeling_function()
def education(record):
    if record['education'] == "Bachelors" or record['education'] == "Masters":
     return income_high
    else:
     return	income_low

In [53]:
#Based on Bi-variate analysis of hours per week and income, we come up with heuristic rule that
# if working hours greater than 40 then income is greater than 50 K otherwise lessthan 50 K
@labeling_function()
def hours_per_week(record):
    if record['hours.per.week'] > 40 or record['hours.per.week'] < 60:
     return income_high
    else:
     return	income_low

In [54]:
@labeling_function()
def work_class(record):
    if record['workclass'] == "Self-emp-inc" or record['workclass'] == "Federal-gov":
     return income_high
    else:
     return	income_low

In [64]:

lfs = [age,education,hours_per_week,work_class]



In [65]:

from snorkel.labeling import PandasLFApplier

In [66]:


applier = PandasLFApplier(lfs=lfs)


L_train = applier.apply(df=x)


100%|██████████| 32561/32561 [00:00<00:00, 61273.89it/s]


In [59]:
L_train

array([[-1,  0,  1,  0],
       [-1,  0,  1,  0],
       [-1,  0,  1,  0],
       ...,
       [ 2,  0,  1,  0],
       [-1,  0,  1,  0],
       [-1,  0,  1,  0]])

In [67]:

from snorkel.labeling.model.label_model import LabelModel


In [68]:

label_model = LabelModel(verbose=False)
label_model.fit(L_train=L_train, n_epochs=1000, seed=100)


100%|██████████| 1000/1000 [00:00<00:00, 3912.36epoch/s]




x['Labels'] = label_model.predict(L=L_train)



In [29]:

x['Labels'] = label_model.predict(L=L_train)

In [69]:
x['Labels']

0        0
1        0
2        0
3        1
4        1
        ..
32556    0
32557    0
32558    1
32559    0
32560    0
Name: Labels, Length: 32561, dtype: int64

In [70]:
x

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,Labels
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,0
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,0
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,0
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,1
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,0
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
